# Config

In [ ]:
# parameters
SAVE_OUTPUT = True
SIZE_PLOTS = (12,10)
YEAR = 2021
LOG = True
#Location of the data
INPUT_DATA_PATH = "../data/raw/"
INPUT_PROCESSED_DATA_PATH = "../data/processed/"
OUTPUT_DATA_PATH = "../data/interim/"
OUTPUT_REPORTS_PATH = "../reports/"


In [2]:
# Parameters
YEAR = 2024
SAVE_OUTPUT = True


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime

from pathlib import Path
import json

# Load data

In [4]:
colnames=['id', 'date', 'weekday', 'hour', 'intensity', 'error'] 
if YEAR == 2024:
    counters_data_1 = pd.read_csv(f"{INPUT_DATA_PATH}/{YEAR}_1_ciclovias.csv", sep=";",names=colnames, header=None)
    counters_data_2 = pd.read_csv(f"{INPUT_DATA_PATH}/{YEAR}_2_ciclovias.csv", sep=";",names=colnames, header=None)
    counters_data = pd.concat([counters_data_1, counters_data_2])
elif YEAR == 2023:
    counters_data = pd.read_csv(f"{INPUT_DATA_PATH}/{YEAR}_ciclovias.csv", sep=";")
else: 
    counters_data = pd.read_csv(f"{INPUT_DATA_PATH}/{YEAR}_ciclovias.csv", sep=";",names=colnames, header=None)
    

# Data management

In [10]:
print(counters_data.shape)
print(counters_data.columns)
print(counters_data.dtypes)
counters_data.head()

(28911552, 6)
Index(['id', 'date', 'weekday', 'hour', 'intensity', 'error'], dtype='object')
id             int64
date          object
weekday       object
hour          object
intensity    float64
error        float64
dtype: object


,id,date,weekday,hour,intensity,error
0,1001,2024-06-01,Sabado,00:00:00.0000000,369.0,0.0
1,1001,2024-06-01,Sabado,00:15:00.0000000,415.0,0.0
2,1001,2024-06-01,Sabado,00:30:00.0000000,330.0,0.0
3,1001,2024-06-01,Sabado,00:45:00.0000000,387.0,0.0
4,1001,2024-06-01,Sabado,01:00:00.0000000,335.0,0.0


## Duplicates

In [ ]:
counters_data = counters_data.drop_duplicates()
len(counters_data["id"].unique())
counters_data.shape

(28911552, 6)

## Errors

### Classify

In [12]:
conditions = [
    (counters_data["error"] == 0.0),
    (counters_data["error"] == 100.0),
    (counters_data["error"].isna()),

    (counters_data["error"] != 0.0) & (counters_data["error"] != 100.0)
]

choices = ["VALID", "INVALID", "UNKNOWN", "PARTIAL"]

counters_data["ERROR"] = np.select(conditions, choices, default="VALID")

observations_per_day = counters_data.groupby(['date', 'ERROR'])['intensity'].count().reset_index(name='observations')

### Calculate metrics

In [ ]:
# print(counters_data["error"].value_counts(dropna=False))
# errors=counters_data[(counters_data["error"]!=0.0)&(counters_data["intensity"].notna())].shape[0]/counters_data.shape[0]

# print(f"Percentage of observations where the couter was malfunctioning: {round(errors,3)}%")

In [ ]:
# counters_data_clean=counters_data[(counters_data["error"]==0.0)|(counters_data["intensity"].isna())].copy()
# counters_data_clean=counters_data_clean.drop(columns=["error"])
counters_data_clean=counters_data


## Get weekday

In [9]:
counters_data_clean['date'] = pd.to_datetime(counters_data_clean['date'])
counters_data_clean["weekday"]=counters_data_clean["date"].dt.day_name()
counters_data_clean["weekday"].value_counts()

NameError: name 'counters_data_clean' is not defined

## Estimate DBT, MBT, AADBT


### DBT


In [12]:
def flag_insuficient_daily_data(df):
    df['duration']=0.25
    df_day = df.groupby(['id', 'date'])['duration'].sum().reset_index()
    df_day=df_day.rename(columns={'duration':'Number_of_hrs_data_collected'})
    print(df_day['Number_of_hrs_data_collected'].describe())

    df=pd.merge(df, df_day, on=['id', 'date'], how='inner')

    flagged_rows = df.loc[(df['Number_of_hrs_data_collected'] <= 23)]
    non_flagged_rows = df.loc[(df['Number_of_hrs_data_collected'] > 23)]
    print('Number of flagged rows:', flagged_rows.shape[0], 'Number of non-flagged rows:', non_flagged_rows.shape[0], 'Total rows:', df.shape[0], 'The sum of flagged and non-flagged rows is equal to the total rows:', flagged_rows.shape[0] + non_flagged_rows.shape[0] == df.shape[0])
    return non_flagged_rows,flagged_rows 


In [13]:
non_flagged_rows,flagged_quarters = flag_insuficient_daily_data(counters_data_clean.copy())


count    301162.0
mean         24.0
std           0.0
min          24.0
25%          24.0
50%          24.0
75%          24.0
max          24.0
Name: Number_of_hrs_data_collected, dtype: float64


Number of flagged rows: 0 Number of non-flagged rows: 28911552 Total rows: 28911552 The sum of flagged and non-flagged rows is equal to the total rows: True


In [14]:
DBT = non_flagged_rows.groupby(['id', 'date','weekday'])['intensity'].sum().reset_index()
DBT=DBT.rename(columns={'intensity':'DBT'})
DBT.head()

,id,date,weekday,DBT
0,1001,2024-01-01,Monday,74746.0
1,1001,2024-01-02,Tuesday,116203.0
2,1001,2024-01-03,Wednesday,123482.0
3,1001,2024-01-04,Thursday,126269.0
4,1001,2024-01-05,Friday,113815.0


### MBT

In [15]:
# def flag_insuficient_monthly_data(df):
#     df['duration']=1

#     df['day']=df['date'].dt.day
#     df['month']=df['date'].dt.month
#     df_day = df.groupby(['id', 'month', 'day'])['duration'].sum().reset_index()
#     df_month = df_day.groupby(['id', 'month'])['day'].count().reset_index()
#     df_month=df_month.rename(columns={'day':'Number_of_days_data_collected'})


#     df=pd.merge(df, df_month, on=['id', 'month'], how='inner')

#     flagged_rows = df.loc[(df['Number_of_days_data_collected'] <= 25)]
#     non_flagged_rows = df.loc[(df['Number_of_days_data_collected'] > 25)]
#     print('Number of flagged rows:', flagged_rows.shape[0], 'Number of non-flagged rows:', non_flagged_rows.shape[0], 'Total rows:', df.shape[0], 'The sum of flagged and non-flagged rows is equal to the total rows:', flagged_rows.shape[0] + non_flagged_rows.shape[0] == df.shape[0])
#     return non_flagged_rows,flagged_rows 


In [16]:
# non_flagged_rows,flagged_days=flag_insuficient_monthly_data(DBT)


In [17]:
# MBT = non_flagged_rows.groupby(['id','month'])['DBT'].sum().reset_index()
# MBT=MBT.rename(columns={'DBT':'MBT'})
# print(MBT.shape)
# MBT.head()


### ABT

In [18]:
# def flag_insuficient_annual_data(df):
#     df['duration']=1

#     df_year = df.groupby(['id'])['duration'].sum().reset_index()
#     df_year=df_year.rename(columns={'duration':'Number_of_days_data_collected'})


#     df=pd.merge(df, df_year, on=['id'], how='inner')

#     flagged_rows = df.loc[(df['Number_of_days_data_collected'] <= 11)]
#     non_flagged_rows = df.loc[(df['Number_of_days_data_collected'] > 11)]
#     print('Number of flagged rows:', flagged_rows.shape[0], 'Number of non-flagged rows:', non_flagged_rows.shape[0], 'Total rows:', df.shape[0], 'The sum of flagged and non-flagged rows is equal to the total rows:', flagged_rows.shape[0] + non_flagged_rows.shape[0] == df.shape[0])
#     return non_flagged_rows,flagged_rows 


In [19]:
# non_flagged_rows,flagged_months=flag_insuficient_annual_data(MBT)


In [20]:
# ABT = non_flagged_rows.groupby(['id'])['MBT'].sum().reset_index()
# ABT=ABT.rename(columns={'MBT':'ABT'})
# print(ABT.shape)
# ABT.head()


### AADBT

In [21]:
AADBT=DBT.groupby(['id'])['DBT'].mean().round().reset_index()
AADBT=AADBT.rename(columns={'DBT':'AADBT'})
AADBT.head()

,id,AADBT
0,1001,112878.0
1,1002,26095.0
2,1003,35131.0
3,1004,837.0
4,1005,29314.0


## Save output

In [22]:
if SAVE_OUTPUT:
    counters_data_clean.to_parquet(f'{OUTPUT_DATA_PATH}/counters_data{YEAR}.parquet')
    # DBT.to_parquet(f'{OUTPUT_DATA_PATH}/DBT{YEAR}.parquet')
    # MBT.to_parquet(f'{OUTPUT_DATA_PATH}/MBT{YEAR}.parquet')
    # ABT.to_parquet(f'{OUTPUT_DATA_PATH}/ABT{YEAR}.parquet')
    AADBT.to_parquet(f'{OUTPUT_DATA_PATH}/AADBT{YEAR}.parquet')
    

In [ ]:
if LOG:
    # Define the new row data
    new_row = {
        'Year': YEAR,
        'Columns': counters_data.shape[1],
        'Rows': counters_data.shape[0],
        'Rows after cleaning': counters_data_clean.shape[0],
        'Deleted rows':  counters_data.shape[0]-counters_data_clean.shape[0],
        # 'Percentage': errors,
        'Bike Counters': counters_data_clean['id'].nunique(),
        'Flagged hours': flagged_quarters.shape[0]/4,
        # 'Flagged days': flagged_days.shape[0],
        # 'Flagged months': flagged_months.shape[0],
    }

    # Convert the new row data to a DataFrame
    new_row_df = pd.DataFrame([new_row])

    # Define the path to the CSV file
    csv_file_path = f"{INPUT_PROCESSED_DATA_PATH}/log_observations.csv"

    # Check if the CSV file exists
    if Path(csv_file_path).is_file():
        # If the file exists, append the new row
        new_row_df.to_csv(csv_file_path, mode='a', header=False, index=False)
    else:
        # If the file does not exist, create it with the new row
        new_row_df.to_csv(csv_file_path, mode='w', header=True, index=False)

In [ ]:
if LOG:
        
    # Calculate the error counts
    error_counts = counters_data["ERROR"].value_counts().to_dict()
    total_errors = sum(error_counts.values())

    # Create a new row with the error counts
    new_error_row = {
        'Year': YEAR,
        'Valid': error_counts.get('VALID', 0),
        'Invalid': error_counts.get('INVALID', 0),
        'Unknown': error_counts.get('UNKNOWN', 0),
        'Partial': error_counts.get('PARTIAL', 0),
        'Total': total_errors,
        'Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }

    # Convert the new row data to a DataFrame
    new_error_row_df = pd.DataFrame([new_error_row])

    # Check if the CSV file exists
    if Path(f"{OUTPUT_REPORTS_PATH}errors.csv").is_file():
        # Load the existing data
        existing_data = pd.read_csv(f"{OUTPUT_REPORTS_PATH}errors.csv")
        
        # Check if a row with the same year exists
        if YEAR in existing_data['Year'].values:
            # Replace the data for the existing year
            existing_data.loc[existing_data['Year'] == YEAR] = new_error_row
        else:
            # Append the new row
            existing_data = pd.concat([existing_data, new_error_row_df], ignore_index=True)
        
        # Save the updated data
        existing_data.to_csv(f"{OUTPUT_REPORTS_PATH}errors.csv", mode='w', header=True, index=False)
    else:
        # If the file does not exist, create it with the new row
        new_error_row_df.to_csv(f"{OUTPUT_REPORTS_PATH}errors.csv", mode='w', header=True, index=False)

## Watermark

In [24]:
!python -m pip install watermark --quiet

In [25]:
%load_ext watermark

In [26]:
%watermark

Last updated: 2025-03-08T14:18:48.888237+01:00

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.25.0

Compiler    : MSC v.1938 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : AMD64 Family 25 Model 68 Stepping 1, AuthenticAMD
CPU cores   : 16
Architecture: 64bit



In [27]:
%watermark --iversions

geopandas: 0.13.2
json     : 2.0.9
numpy    : 1.24.4
pandas   : 2.0.3



In [28]:
!lsb_release -a

"lsb_release" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
